In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
import pickle
import numpy as np
import time
import datetime
%load_ext tensorboard

In [3]:
# load dataset and labels
X_train = pickle.load(open('pickle_data/X_train.pickle','rb'))
y_train = pickle.load(open('pickle_data/y_train.pickle','rb'))

# load validation dataset and labels
X_test = pickle.load(open('pickle_data/X_test.pickle', 'rb'))
y_test = pickle.load(open('pickle_data/y_test.pickle', 'rb'))

print('X_train.shape =', X_train.shape)
print('y_train.shape =', y_train.shape)
print('X_test.shape =', X_test.shape)
print('y_test.shape =', y_test.shape)

X_train.shape = (2320, 50, 50, 3)
y_train.shape = (2320,)
X_test.shape = (258, 50, 50, 3)
y_test.shape = (258,)


In [6]:
# normalize pixel data
X_train = X_train/255.0
X_test = X_test/255.0

In [7]:
def create_model(conv_layer, layer_size, dense_layer):
    # create a new log of the model to analyze results
    tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

    # define model
    model = Sequential()

    # 1st convolution
    model.add(Conv2D(layer_size, (3,3), input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    # add additional conv layers
    for _ in range(conv_layer-1):
        # convolution
        model.add(Conv2D(layer_size, (3,3)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        # Max-Pool to reduce tensor size while retaining distict features
        model.add(MaxPooling2D(pool_size=(2,2)))

    # flatten model for fully connected layers
    model.add(Flatten())

    # add dense layers
    for _ in range(dense_layer):
        # fully-connected
        model.add(Dense(layer_size))
        model.add(Activation('relu'))

    # final fully-connected layer (5 matches number of Pokemon categories)
    model.add(Dense(5))
    model.add(Activation('softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    # fit the model to the training data
    model.fit(X_train, y_train, batch_size=16, epochs=20, validation_split=0.2, callbacks=[tensorboard])
    
    return model

In [ ]:
# define hyperparameters
conv_layer = 14
layer_size = 64
dense_layer = 2

# name of model to keep track of changing model architectures
NAME = '{}-conv-{}-nodes-{}-dense-{}'.format(conv_layer, layer_size, dense_layer, int(time.time()))
    
# create model and fit it
model = create_model(conv_layer, layer_size, dense_layer)

In [11]:
# evaluate the model
scores = model.evaluate(X_validation, y_validation, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 86.05%


In [12]:
# save the model
model.save('models/' + NAME)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: models/4-conv-128-nodes-1-dense-1575226904/assets
